# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv("../output_data/cities.csv")
city_data_df.head()


,City_ID,City,Lat,Lon,Temp,Humidity,Cloudiness,Windspeed,Country
0,0,Nikolskoye,59.7035,30.7861,33.26,80,90,6.71,RU
1,1,Qaanaaq,77.4840,-69.3632,12.45,58,100,9.06,GL
2,2,Port Hedland,-20.3167,118.5667,78.80,83,9,13.80,AU
3,3,Lüderitz,-26.6481,15.1594,65.37,83,0,8.28,NaN
4,4,Butaritari,3.0707,172.7902,81.14,77,54,14.38,KI


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations= city_data_df[['Lat','Lon']]
humidity= city_data_df["Humidity"]
fig= gmaps.figure()
heatmap = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)
fig.add_layer(heatmap)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
city_narrowed_df= city_data_df.loc[(city_data_df["Temp"] < 80) & (city_data_df["Temp"] > 70)\
                                  & (city_data_df["Windspeed"] < 10)\
                                  & (city_data_df["Cloudiness"] == 0)].dropna()
city_narrowed_df

,City_ID,City,Lat,Lon,Temp,Humidity,Cloudiness,Windspeed,Country
32,34,Warangal,18.0000,79.5833,73.60,18,0,6.02,IN
93,95,Pisco,-13.7000,-76.2167,75.11,78,0,6.91,PE
114,116,Salalah,17.0151,54.0924,75.20,69,0,2.30,OM
135,137,Umluj,25.0213,37.2685,70.54,49,0,4.97,SA
186,188,Dingle,10.9995,122.6711,75.87,90,0,5.75,PH
209,211,Bharūch,21.7000,72.9667,73.08,47,0,3.44,IN
212,214,Saldanha,-33.0117,17.9442,77.00,47,0,8.05,ZA
221,224,Sūrandai,8.9667,77.4000,74.23,71,0,1.79,IN
287,292,Am Timan,11.0297,20.2827,78.15,19,0,4.45,TD
333,338,Porbandar,21.6422,69.6093,71.51,89,0,6.38,IN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = city_narrowed_df
paramaters = {"radius": 5000, "types": "lodging", "key": g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lon= row["Lon"]
    paramaters["location"] = f"{lat}, {lon}"
    
    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    print(f"Retrieving Results for Indes {index}: {row['City']}.")
    response = requests.get(base_url, params=paramaters).json()
    results= response['results']
    
    try:
        print(f"Closest Hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result...skipping.")

Retrieving Results for Indes 32: Warangal.
Closest Hotel is OYO 46557 Hotel Dolphin.
Retrieving Results for Indes 93: Pisco.
Closest Hotel is Hotel San Isidro Oficial.
Retrieving Results for Indes 114: Salalah.
Closest Hotel is HAMDAN PLAZA HOTEL SALALAH.
Retrieving Results for Indes 135: Umluj.
Closest Hotel is Moon light Furnished Units.
Retrieving Results for Indes 186: Dingle.
Closest Hotel is SEGAYA PROPERTY.
Retrieving Results for Indes 209: Bharūch.
Closest Hotel is Krishna Guest House.
Retrieving Results for Indes 212: Saldanha.
Closest Hotel is Blue Bay Lodge.
Retrieving Results for Indes 221: Sūrandai.
Closest Hotel is Mahalakshmi marriage mahal.
Retrieving Results for Indes 287: Am Timan.
Closest Hotel is Palais du chef Canton Salamt.
Retrieving Results for Indes 333: Porbandar.
Closest Hotel is Lords Inn Porbandar.
Retrieving Results for Indes 386: Kununurra.
Closest Hotel is Hotel Kununurra.
Retrieving Results for Indes 389: Bonthe.
Closest Hotel is Bonthe Holiday Village.

In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [8]:
# Add marker layer ontop of heat map
# hotel_layer = gmaps.symbol_layer(
#     locations, fill_color= 'rgba(0, 150, 0, 0.4)',
#     stroke_color ='rgna(0, 0, 150, 0.4)', scale=2,
#     info_box_content=hotel_info

# Display figure
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig
    

Figure(layout=FigureLayout(height='420px'))